In [24]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display # to display plots on jypter notebooks

### Linear Transformations

In [25]:
animations = [] # Store animations

### MAIN FUNCTION ###
def transform(ascii_art, matrix):
    # Convert ASCII art to coordinates
    original_coords = _ascii_art_to_coords(ascii_art)
    
    # Apply the linear transformation
    transformed_coords = _apply_linear_transform(original_coords, matrix)
    
    # Create an animation of the linear transformation
    anim = _animate_graph(original_coords, transformed_coords, matrix)
    
    return anim

In [26]:
### HELPER FUNCTIONS ###
def _ascii_art_to_coords(ascii_art):
    """Convert ascii art into a list of coordinates for each character"""
    coords = [] #  Initialize coordinates of ascii characters
    max_y = len(ascii_art) - 1 #  Store height of ascii art
    for y_idx, line in enumerate(ascii_art): #  loop thru each row of text
        y = max_y - y_idx #  to imitate a real graph, the top of ascii art will be the highest y value
        for x, char in enumerate(line): #  loop thru each character in row
            if char != ' ': #  if not a space, add (x, y, char) tuple to list
                coords.append((x, y, char))
    return coords #  return list of (x, y, char) tuples
                
def _apply_linear_transform(coords, matrix):
    """Apply a linear transformation to each point using a given matrix"""
    new_coords = [] 
    for x, y, char in coords: #  loop thru each point in coords list
        vector = np.array([x, y]) #  create a vector from coordinates
        new_pos = matrix @ vector #  @ operator used for matrix multiplication
        new_coords.append((new_pos[0], new_pos[1], char)) #  add transformed coords to list, return them
    return new_coords

def _animate_graph(original, transformed, matrix=None):
    """Create an interactive animation of the linear transformation"""
    # Extract both the original and transformed coordinates
    ox = np.array([x for x, y, c in original])
    oy = np.array([y for x, y, c in original])
    tx = np.array([x for x, y, c in transformed])
    ty = np.array([y for x, y, c in transformed])

    # Setup scatter plot
    # Plot original points as blue squares
    fig, ax = plt.subplots(figsize=(8, 6))
    scat = ax.scatter(ox, oy, c='blue', marker='s')
    
    # Make a single array of both original+transformed points
    all_x = np.concatenate([ox, tx])
    all_y = np.concatenate([oy, ty])
    # Use array to set x,y limits to fit points into scatter plot
    ax.set_xlim(all_x.min() - 1, all_x.max() + 1)
    ax.set_ylim(all_y.min() - 1, all_y.max() + 1)
    ax.set_aspect('equal') # 1:1 aspect ratio
    
    ax.grid(True)
    ax.set_title("Linear Transformation of ASCII Art")

    # Compute eigenvectors (the vectors that do not change in magnitude nor direction under a linear transformation)
    if matrix is not None: # If the matrix exists
        eigvals, eigvecs = np.linalg.eig(matrix) 
        eigvecs = eigvecs / np.linalg.norm(eigvecs, axis=0) #  normalize vectors (set length to 1)
        quiv = ax.quiver( 
            [0]*len(eigvals), [0]*len(eigvals),
            eigvecs[0,:], eigvecs[1,:],
            angles='xy', scale_units='xy', scale=1, color='red' #  draw eigenvectors as red arrows
        )
        
        # Add eigenvalue labels
        labels = []
        for i in range(len(eigvals)):
            label = ax.text(eigvecs[0,i]*3, eigvecs[1,i]*3, f"λ={eigvals[i]:.2f}", color='red')
            labels.append(label)

    frames = 60 # how smooth the animation is

    def update(frame):
        """Updates the scatterplot once per frame"""
        t = frame / (frames - 1) #  tracks the progression of animation from 0-1
        # Interpolate all points
        x = (1-t) * ox + t * tx
        y = (1-t) * oy + t * ty
        scat.set_offsets(np.c_[x, y])
        scat.set_color(plt.cm.viridis(t))

        if matrix is not None:
            # Scale eigenvectors by eigenvalues
            U = eigvecs * ((1-t) + t * eigvals)  # linear interpolation
            quiv.set_UVC(U[0,:], U[1,:])
        return (scat, quiv)

    # Create the animation
    anim = FuncAnimation(fig, update, frames=frames, interval=50, blit=False, repeat=True)
    
    plt.close(fig) # prevent duplicates
    
    return anim

In [27]:
# Example 1
ascii_art = [
    "     *     ",
    "    ***    ",
    "   *****   ",
    "  *******  ",
    " ********* ",
    "    ***    ",
    "    ***    "
]

matrix1 = np.array([
    [2, 1],
    [1, 3]
])

anim1 = transform(ascii_art, matrix1)

display(HTML(anim1.to_jshtml()))

In [28]:
# Example 2
ascii_art = [
    "     *     ",
    "    ***    ",
    "   *****   ",
    "  *******  ",
    " ********* ",
    "    ***    ",
    "    ***    "
]

matrix2 = np.array([
    [1, 1.1],
    [0, 1]
])

anim2 = transform(ascii_art, matrix2)

display(HTML(anim2.to_jshtml()))

### Connecting to Eigenvalues and Eigenvectors

In [29]:
def eigen(matrix):
    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(matrix)

    # Output results
    print("Matrix:")
    print(matrix)

    # Display eigenvectors alongside their eigenvalues
    print("\nEigenvalue-Eigenvector pairs:")
    for i in range(len(eigenvalues)):
        print(f"λ = {eigenvalues[i]:.4f}, v = any multiple of {np.round(eigenvectors[:, i],4)}")

In [30]:
eigen(matrix1)

Matrix:
[[2 1]
 [1 3]]

Eigenvalue-Eigenvector pairs:
λ = 1.3820, v = any multiple of [-0.8507  0.5257]
λ = 3.6180, v = any multiple of [-0.5257 -0.8507]


In [31]:
eigen(matrix2)

Matrix:
[[1.  1.1]
 [0.  1. ]]

Eigenvalue-Eigenvector pairs:
λ = 1.0000, v = any multiple of [1. 0.]
λ = 1.0000, v = any multiple of [-1.  0.]


### Bonus Materials

In [32]:
import math
# Sample Linear Transformations you can do

# Rotation (counterclockwise)
theta_deg = 45
theta = math.radians(theta_deg)
rotation_matrix = np.array([
    [math.cos(theta), -math.sin(theta)],
    [math.sin(theta),  math.cos(theta)]
])

# Scaling
scaling_matrix = np.array([
    [2, 0],
    [0, 1]
])

# Shearing (x-direction)
shear_matrix = np.array([
    [1, 0.5],
    [0, 1]
])